In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def play_video(video, n):
    
    cap = cv2.VideoCapture(video)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("#######################################################################")
    print("sample_" + str(n))
    i = 0
    while(True):

        ret, frame = cap.read()
       # print(ret)# ret = 1 if the video is captured; frame is the image
        if(ret == False):
            break
            
        img = frame
        if(i < 2 or i > frame_count - 5):
            plt.imshow(frame)
            plt.show()
        i = i + 1
#         # Display the resulting image
#         cv2.imshow('Video Capture',img)
#         if cv2.waitKey(1) & 0xFF == ord('q'):  # press q to quit
#             break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()



In [9]:
# import os

# for i in range(75):
#     if(os.path.exists("rudrabha/front/sample_"+str(i+11)+".avi")):
#         play_video("rudrabha/front/sample_"+str(i+11)+".avi", i)
#     else:
#         print('n')

In [11]:
############################ Mapping Projected video to original video ###############

def print_callibrate_corresponding_videos(video1, video2, n):
    
    cap1 = cv2.VideoCapture(video1)
    frame_count1 = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    
    cap2 = cv2.VideoCapture(video2)
    frame_count2 = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print("#######################################################################")
    print("sample_" + str(n), frame_count1, frame_count2)
    
    i = 0
    while(True):

        ret1, frame1 = cap1.read() 
        ret2, frame2 = cap2.read() 
        
        print(frame1.shape, frame2.shape)
        plt.imshow(frame1)
        plt.show()
        
        plt.imshow(frame2)
        plt.show()
        
        break
        
        

In [13]:
# for i in range(75):
#     print_callibrate_corresponding_videos("rudrabha/front/sample_"+str(i+11)+".avi", 
#                                           "original_samples/trip"+str(i+1)+"_out_hist.avi" , i)

In [19]:
def count_frames(video):
    count = 0
    vid = cv2.VideoCapture(video)    
    while(True):
        ret, frame = vid.read()
        if(ret == False):
            break
        count += 1
        
    return count

In [48]:
import pandas as pd

def callibrate_corresponding_videos(video1, video2, video3, textfile, output_vid1, output_vid2, output_vid3, 
                                    out_textfile, n1):
    
    print("############################################################3")
    print("sample_" +str(n1))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
    file1 = pd.read_csv(textfile, header = None).values
    print(textfile,file1.shape)
    
    arr = [];
    
    #################################################################################################
    out1 = cv2.VideoWriter(output_vid1, fourcc, 25, (1440,1080)) ### driver
    out2 = cv2.VideoWriter(output_vid2, fourcc, 25, (1440,1080)) ### road
    out3 = cv2.VideoWriter(output_vid3, fourcc, 25, (1920, 1080)) ### original
    
    #################################################################################################
    cap1 = cv2.VideoCapture(video1)###### driver ###################
    frame_count1 = count_frames(video1)
    
    cap2 = cv2.VideoCapture(video2)###### road ####################
    frame_count2 = count_frames(video2)
    
    cap3 = cv2.VideoCapture(video3)###### Original ####################
    frame_count3 = count_frames(video3)
    
    #print(frame_count1, frame_count2, frame_count3)

    #################################################################################################
    i = 0; count = 0; flag =0

    if(frame_count2 == frame_count3):
        flag =1
        n = frame_count2
        first = frame_count2
        second = frame_count3

    elif(frame_count3 < frame_count2):
        n = frame_count2
        first = frame_count2
        second = frame_count3

    elif(frame_count3 > frame_count2):
        n = frame_count3
        first = frame_count3
        second = frame_count2

    diff = first - second
    if(diff != 0):
        drop = np.round(first/diff)
    print(n, drop, first, second, diff)
    k =0
    #print(file1)
    while(i < n and flag == 0):

        if(diff != 0 and i%drop == 0 ):
           # print(i)
            if(frame_count2 > frame_count3):
                ret2, frame2 = cap2.read()
                ret1, frame1 = cap1.read()
                
            else:
                ret3, frame3 = cap3.read()
               # print(i)
                k +=1
        else:
            count += 1
           # print(k)
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            ret3, frame3 = cap3.read()
            if(ret2 == False or ret3 == False):
                break
            arr.append(file1[k])
            k +=1

            #print(frame1.shape, frame2.shape, frame3.shape)
          #  frame2 = cv2.resize(frame2, (692, 460))
          #  print(frame1.shape, frame2.shape)
            #print(ret1, ret2)
#             if(ret2 == False or ret3 == False):
#                 break
         #   frame3 = np.concatenate((frame1, frame2), axis =1)
            out1.write(frame1)
            out2.write(frame2)
            out3.write(frame3)

        i += 1
        
    print(first, second, count)
    
    arr = np.array(arr)
    np.save(out_textfile, arr)
        
    out1.release(); out2.release(); out3.release()
    cap1.release(); cap2.release(); cap3.release()
        
    

In [54]:
for i in range(75,76):
    video1 = "rudrabha/back/sample_"+str(i+11)+".avi"            
    video2 = "rudrabha/front/sample_"+str(i+11)+".avi"
    video3 = "original_samples/trip"+str(i+1)+"_out_hist.avi"
    out_video1 = "rudrabha_out/driver_side/sample_"+str(i+11)+".avi"
    out_video2 = "rudrabha_out/road_side/sample_"+str(i+11)+".avi"
    out_video3 = "rudrabha_out/original_road_side/sample_"+str(i+1)+".avi"
    out_textfile = "rudrabha_out/original_road_side/sample_"+str(i+1)+".npy"

    textfile = "original_samples/trip"+str(i+1)+"_out.txt"

    callibrate_corresponding_videos(video1, video2, video3, textfile, out_video1, out_video2, out_video3, out_textfile, i)

############################################################3
sample_75
original_samples/trip76_out.txt (91, 5)
170 2.0 170 91 79
170 91 85


In [59]:
for i in range(88, 104):
    video1 = "rudrabha/back/sample_"+str(i+11)+".avi"            
    video2 = "rudrabha/front/sample_"+str(i+11)+".avi"
    video3 = "original_samples/trip_"+str(i+1)+"_hist_out.avi"
    out_video1 = "rudrabha_out/driver_side/sample_"+str(i+11)+".avi"
    out_video2 = "rudrabha_out/road_side/sample_"+str(i+11)+".avi"
    out_video3 = "rudrabha_out/original_road_side/sample_"+str(i+1)+".avi"
    out_textfile = "rudrabha_out/original_road_side/sample_"+str(i+1)+".npy"

    textfile = "original_samples/trip_"+str(i+1)+"_hist_out.txt"

    callibrate_corresponding_videos(video1, video2, video3, textfile, out_video1, out_video2, out_video3, out_textfile, i)

############################################################3
sample_88
original_samples/trip_89_hist_out.txt (134, 5)
144 14.0 144 134 10
144 134 133
############################################################3
sample_89
original_samples/trip_90_hist_out.txt (114, 5)
114 16.0 114 107 7
114 107 106
############################################################3
sample_90
original_samples/trip_91_hist_out.txt (163, 5)
163 3.0 163 114 49
163 114 108
############################################################3
sample_91
original_samples/trip_92_hist_out.txt (102, 5)
102 6.0 102 85 17
102 85 85
############################################################3
sample_92
original_samples/trip_93_hist_out.txt (130, 5)
130 3.0 130 89 41
130 89 86
############################################################3
sample_93
original_samples/trip_94_hist_out.txt (74, 5)
74 7.0 74 64 10
74 64 63
############################################################3
sample_94
original_samples/trip_95_hist_out.txt (9

In [108]:
def sample_videos50(video1, video2, video3, textfile, output_vid1, output_vid2, output_vid3, 
                                    out_textfile, n1):
    
    print("############################################################3")
    print("sample_" +str(n1))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    file1 = np.load(textfile)
    print(textfile,file1.shape)
    
    arr = [];
    
    #################################################################################################
    out1 = cv2.VideoWriter(output_vid1, fourcc, 25, (1440,1080)) ### driver
    out2 = cv2.VideoWriter(output_vid2, fourcc, 25, (1440,1080)) ### road
    out3 = cv2.VideoWriter(output_vid3, fourcc, 25, (1920, 1080)) ### original
    
    #################################################################################################
    cap1 = cv2.VideoCapture(video1)###### driver ###################
    frame_count1 = count_frames(video1)
    
    cap2 = cv2.VideoCapture(video2)###### road ####################
    frame_count2 = count_frames(video2)
    
    cap3 = cv2.VideoCapture(video3)###### Original ####################
    frame_count3 = count_frames(video3)
    
    #################################################################################################
    i = 0; count = 0; flag =0
    
    n = np.min([frame_count1, frame_count2, frame_count3])
    drop = np.round(n/(n - 50))
    drop = 2

    print(n1, frame_count1, frame_count2, frame_count3, drop)
    k =0
    #print(file1)
    while(i < n and flag == 0):

        if(i%drop == 0  and frame_count1 > 50):
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            ret3, frame3 = cap3.read()
            k +=1
        else:
            count += 1
           # print(k)
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            ret3, frame3 = cap3.read()
            if(ret2 == False or ret3 == False):
                break
            arr.append(file1[k])
            k +=1

            out1.write(frame1)
            out2.write(frame2)
            out3.write(frame3)

        i += 1
        
    print(frame_count1, frame_count2, frame_count3, count)
    
    arr = np.array(arr)
    np.save(out_textfile, arr)
        
    out1.release(); out2.release(); out3.release()
    cap1.release(); cap2.release(); cap3.release()
        
    

In [109]:
for i in range(75):
    video1 = "rudrabha_out/driver_side/sample_"+str(i+11)+".avi"            
    video2 = "rudrabha_out/road_side/sample_"+str(i+11)+".avi"
    video3 = "rudrabha_out/original_road_side/sample_"+str(i+1)+".avi"
    out_video1 = "rudrabha_out50/driver_side/sample_"+str(i+11)+".avi"
    out_video2 = "rudrabha_out50/road_side/sample_"+str(i+11)+".avi"
    out_video3 = "rudrabha_out50/original_road_side/sample_"+str(i+1)+".avi"
    out_textfile = "rudrabha_out50/original_road_side/sample_"+str(i+1)+".npy"

    textfile = "rudrabha_out/original_road_side/sample_"+str(i+1)+".npy"

    sample_videos50(video1, video2, video3, textfile, out_video1, out_video2, out_video3, out_textfile, i)

############################################################3
sample_0
rudrabha_out/original_road_side/sample_1.npy (63, 5)
0 63 63 63 2
63 63 63 31
############################################################3
sample_1
rudrabha_out/original_road_side/sample_2.npy (112, 5)
1 112 112 112 2
112 112 112 56
############################################################3
sample_2
rudrabha_out/original_road_side/sample_3.npy (86, 5)
2 86 86 86 2
86 86 86 43
############################################################3
sample_3
rudrabha_out/original_road_side/sample_4.npy (88, 5)
3 88 88 88 2
88 88 88 44
############################################################3
sample_4
rudrabha_out/original_road_side/sample_5.npy (77, 5)
4 77 77 77 2
77 77 77 38
############################################################3
sample_5
rudrabha_out/original_road_side/sample_6.npy (97, 5)
5 97 97 97 2
97 97 97 48
############################################################3
sample_6
rudrabha_out/original_road_s

52 114 114 114 2
114 114 114 57
############################################################3
sample_53
rudrabha_out/original_road_side/sample_54.npy (87, 5)
53 87 87 87 2
87 87 87 43
############################################################3
sample_54
rudrabha_out/original_road_side/sample_55.npy (47, 5)
54 47 47 47 2
47 47 47 47
############################################################3
sample_55
rudrabha_out/original_road_side/sample_56.npy (288, 5)
55 288 288 288 2
288 288 288 144
############################################################3
sample_56
rudrabha_out/original_road_side/sample_57.npy (81, 5)
56 81 81 81 2
81 81 81 40
############################################################3
sample_57
rudrabha_out/original_road_side/sample_58.npy (107, 5)
57 107 107 107 2
107 107 107 53
############################################################3
sample_58
rudrabha_out/original_road_side/sample_59.npy (60, 5)
58 60 60 60 2
60 60 60 30
#########################################

FileNotFoundError: [Errno 2] No such file or directory: 'rudrabha_out/original_road_side/sample_69.npy'

In [94]:
import os
i =0
os.path.exists("rudrabha/back/sample_"+str(i+11)+".avi"     )

True